# Model Relationships

## Learning Objectives
- Understand different types of database relationships
- Learn how to define relationships in SQLAlchemy models
- Master one-to-many relationships with foreign keys
- Understand many-to-many relationships with association tables
- Learn about relationship loading strategies
- Practice querying related data

## What You'll Learn
- One-to-many relationships (User → Posts)
- Many-to-many relationships (Students ↔ Courses)
- Foreign keys and relationship definitions
- Back references and bidirectional relationships
- Lazy loading vs eager loading
- Querying related objects efficiently


## Types of Database Relationships

### 1. One-to-Many (1:N)
- **Example**: One User has many Posts
- **Implementation**: Foreign key in the "many" table
- **SQLAlchemy**: `relationship()` with `foreign_keys`

### 2. Many-to-Many (M:N)
- **Example**: Students can enroll in many Courses, Courses can have many Students
- **Implementation**: Association table with foreign keys to both tables
- **SQLAlchemy**: `relationship()` with `secondary` parameter

### 3. One-to-One (1:1)
- **Example**: One User has one Profile
- **Implementation**: Foreign key with unique constraint
- **SQLAlchemy**: `relationship()` with `uselist=False`


## 1. One-to-Many Relationships

Let's start with the most common relationship type:


In [ ]:
# Setup
from sqlalchemy import create_engine, Column, Integer, String, DateTime, ForeignKey, Text
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker, relationship
from datetime import datetime

# Create database engine
engine = create_engine('sqlite:///relationships.db', echo=True)
Base = declarative_base()
Session = sessionmaker(bind=engine)

print("SQLAlchemy setup complete!")


In [ ]:
# Define User model (Parent)
class User(Base):
    __tablename__ = 'users'
    
    id = Column(Integer, primary_key=True)
    username = Column(String(50), unique=True, nullable=False)
    email = Column(String(100), unique=True, nullable=False)
    created_at = Column(DateTime, default=datetime.utcnow)
    
    # One-to-many relationship: One user has many posts
    posts = relationship("Post", back_populates="author", cascade="all, delete-orphan")
    
    def __repr__(self):
        return f"<User(username='{self.username}', email='{self.email}')>"

# Define Post model (Child)
class Post(Base):
    __tablename__ = 'posts'
    
    id = Column(Integer, primary_key=True)
    title = Column(String(200), nullable=False)
    content = Column(Text, nullable=False)
    created_at = Column(DateTime, default=datetime.utcnow)
    
    # Foreign key to users table
    author_id = Column(Integer, ForeignKey('users.id'), nullable=False)
    
    # Many-to-one relationship: Many posts belong to one user
    author = relationship("User", back_populates="posts")
    
    def __repr__(self):
        return f"<Post(title='{self.title}', author_id={self.author_id})>"

print("User and Post models defined with one-to-many relationship!")


In [ ]:
# Create tables
Base.metadata.create_all(engine)

# Create session
session = Session()

# Create users and posts
user1 = User(username='alice', email='alice@example.com')
user2 = User(username='bob', email='bob@example.com')

# Add users to session
session.add(user1)
session.add(user2)
session.commit()

print("Users created successfully!")

# Create posts for users
post1 = Post(title='My First Post', content='This is my first blog post!', author_id=user1.id)
post2 = Post(title='Python Tips', content='Here are some Python programming tips.', author_id=user1.id)
post3 = Post(title='Database Design', content='Learn about database relationships.', author_id=user2.id)

# Add posts to session
session.add_all([post1, post2, post3])
session.commit()

print("Posts created successfully!")


In [ ]:
# Querying related data

# Get a user and their posts
user = session.query(User).filter(User.username == 'alice').first()
print(f"User: {user}")
print(f"User's posts: {user.posts}")

# Get a post and its author
post = session.query(Post).filter(Post.title == 'My First Post').first()
print(f"\nPost: {post}")
print(f"Post's author: {post.author}")

# Get all posts with their authors
posts_with_authors = session.query(Post).all()
print(f"\nAll posts with authors:")
for post in posts_with_authors:
    print(f"  {post.title} by {post.author.username}")


## 2. Many-to-Many Relationships

Now let's explore many-to-many relationships using Students and Courses:


In [ ]:
# Define Student model
class Student(Base):
    __tablename__ = 'students'
    
    id = Column(Integer, primary_key=True)
    name = Column(String(100), nullable=False)
    email = Column(String(100), unique=True, nullable=False)
    
    # Many-to-many relationship with courses
    courses = relationship("Course", secondary="enrollments", back_populates="students")
    
    def __repr__(self):
        return f"<Student(name='{self.name}', email='{self.email}')>"

# Define Course model
class Course(Base):
    __tablename__ = 'courses'
    
    id = Column(Integer, primary_key=True)
    title = Column(String(200), nullable=False)
    description = Column(Text)
    credits = Column(Integer, default=3)
    
    # Many-to-many relationship with students
    students = relationship("Student", secondary="enrollments", back_populates="courses")
    
    def __repr__(self):
        return f"<Course(title='{self.title}', credits={self.credits})>"

# Define association table for many-to-many relationship
from sqlalchemy import Table

enrollments = Table('enrollments', Base.metadata,
    Column('student_id', Integer, ForeignKey('students.id'), primary_key=True),
    Column('course_id', Integer, ForeignKey('courses.id'), primary_key=True),
    Column('enrolled_at', DateTime, default=datetime.utcnow)
)

print("Student and Course models with many-to-many relationship defined!")


In [ ]:
# Create new tables for many-to-many relationship
Base.metadata.create_all(engine)

# Create students and courses
student1 = Student(name='Alice Johnson', email='alice@university.edu')
student2 = Student(name='Bob Smith', email='bob@university.edu')
student3 = Student(name='Charlie Brown', email='charlie@university.edu')

course1 = Course(title='Python Programming', description='Learn Python fundamentals', credits=4)
course2 = Course(title='Database Design', description='Database concepts and SQL', credits=3)
course3 = Course(title='Web Development', description='HTML, CSS, and JavaScript', credits=4)

# Add to session
session.add_all([student1, student2, student3, course1, course2, course3])
session.commit()

print("Students and courses created successfully!")

# Create many-to-many relationships (enrollments)
student1.courses.append(course1)  # Alice enrolls in Python
student1.courses.append(course2)  # Alice enrolls in Database Design
student2.courses.append(course1)  # Bob enrolls in Python
student2.courses.append(course3)  # Bob enrolls in Web Development
student3.courses.append(course2)  # Charlie enrolls in Database Design
student3.courses.append(course3)  # Charlie enrolls in Web Development

session.commit()
print("Enrollments created successfully!")


In [ ]:
# Querying many-to-many relationships

# Get a student and their courses
student = session.query(Student).filter(Student.name == 'Alice Johnson').first()
print(f"Student: {student}")
print(f"Alice's courses: {student.courses}")

# Get a course and its students
course = session.query(Course).filter(Course.title == 'Python Programming').first()
print(f"\nCourse: {course}")
print(f"Python Programming students: {course.students}")

# Get all students with their courses
all_students = session.query(Student).all()
print(f"\nAll students and their courses:")
for student in all_students:
    print(f"  {student.name}: {[course.title for course in student.courses]}")

# Get all courses with their students
all_courses = session.query(Course).all()
print(f"\nAll courses and their students:")
for course in all_courses:
    print(f"  {course.title}: {[student.name for student in course.students]}")


## 3. One-to-One Relationships

Let's explore one-to-one relationships with User and Profile:


In [ ]:
# Define Profile model for one-to-one relationship
class Profile(Base):
    __tablename__ = 'profiles'
    
    id = Column(Integer, primary_key=True)
    first_name = Column(String(50))
    last_name = Column(String(50))
    bio = Column(Text)
    phone = Column(String(20))
    
    # Foreign key to users table
    user_id = Column(Integer, ForeignKey('users.id'), unique=True, nullable=False)
    
    # One-to-one relationship: One profile belongs to one user
    user = relationship("User", backref="profile", uselist=False)
    
    def __repr__(self):
        return f"<Profile(first_name='{self.first_name}', last_name='{self.last_name}')>"

print("Profile model with one-to-one relationship defined!")


In [ ]:
# Create profiles table
Base.metadata.create_all(engine)

# Create profiles for existing users
profile1 = Profile(
    first_name='Alice',
    last_name='Johnson',
    bio='Software developer passionate about Python',
    phone='555-0123',
    user_id=user1.id
)

profile2 = Profile(
    first_name='Bob',
    last_name='Smith',
    bio='Database administrator and SQL expert',
    phone='555-0456',
    user_id=user2.id
)

# Add profiles to session
session.add_all([profile1, profile2])
session.commit()

print("Profiles created successfully!")

# Querying one-to-one relationships
user = session.query(User).filter(User.username == 'alice').first()
print(f"User: {user}")
print(f"User's profile: {user.profile}")

profile = session.query(Profile).filter(Profile.first_name == 'Alice').first()
print(f"\nProfile: {profile}")
print(f"Profile's user: {profile.user}")


## 4. Relationship Loading Strategies

Understanding how SQLAlchemy loads related data:


In [ ]:
from sqlalchemy.orm import joinedload, subqueryload

# Lazy loading (default) - loads related data when accessed
print("=== Lazy Loading ===")
user = session.query(User).filter(User.username == 'alice').first()
print(f"User loaded: {user}")
# Posts are not loaded yet
print(f"Posts loaded: {user.posts}")  # This triggers a database query

# Eager loading with joinedload - loads related data in a single query
print("\n=== Eager Loading (joinedload) ===")
user_with_posts = session.query(User).options(joinedload(User.posts)).filter(User.username == 'alice').first()
print(f"User with posts loaded: {user_with_posts}")
print(f"Posts: {user_with_posts.posts}")  # No additional query needed

# Eager loading with subqueryload - loads related data in a separate query
print("\n=== Eager Loading (subqueryload) ===")
user_with_posts_sub = session.query(User).options(subqueryload(User.posts)).filter(User.username == 'alice').first()
print(f"User with posts loaded: {user_with_posts_sub}")
print(f"Posts: {user_with_posts_sub.posts}")  # No additional query needed


## 5. Advanced Relationship Queries

Let's explore more complex queries with relationships:


In [ ]:
# Find users who have posts
users_with_posts = session.query(User).join(Post).distinct().all()
print("Users who have posts:")
for user in users_with_posts:
    print(f"  {user.username}")

# Find users with more than 1 post
from sqlalchemy import func
users_with_multiple_posts = session.query(User).join(Post).group_by(User.id).having(func.count(Post.id) > 1).all()
print(f"\nUsers with multiple posts:")
for user in users_with_multiple_posts:
    print(f"  {user.username}")

# Find courses with more than 2 students
courses_with_many_students = session.query(Course).join(enrollments).group_by(Course.id).having(func.count(enrollments.c.student_id) > 2).all()
print(f"\nCourses with more than 2 students:")
for course in courses_with_many_students:
    print(f"  {course.title}")

# Find students enrolled in Python Programming
python_students = session.query(Student).join(enrollments).join(Course).filter(Course.title == 'Python Programming').all()
print(f"\nStudents enrolled in Python Programming:")
for student in python_students:
    print(f"  {student.name}")


In [ ]:
# Close session
session.close()

print("\n" + "="*50)
print("Relationship examples completed successfully!")
print("="*50)


## Summary

You've learned about:

1. **One-to-Many Relationships** - User → Posts with foreign keys
2. **Many-to-Many Relationships** - Students ↔ Courses with association tables  
3. **One-to-One Relationships** - User ↔ Profile with unique foreign keys
4. **Relationship Loading** - Lazy vs eager loading strategies
5. **Advanced Queries** - Complex queries with joins and aggregations

## Key Takeaways

- **Foreign Keys**: Use `ForeignKey()` to establish relationships
- **Relationship Definitions**: Use `relationship()` with appropriate parameters
- **Association Tables**: Use `Table()` for many-to-many relationships
- **Loading Strategies**: Choose between lazy, joinedload, and subqueryload
- **Query Optimization**: Use joins and eager loading to reduce database queries
- **Bidirectional Relationships**: Use `back_populates` for proper two-way relationships

Ready to practice? Move on to the exercise notebook!


## Summary

You've learned about:

1. **One-to-Many Relationships** - User → Posts with foreign keys
2. **Many-to-Many Relationships** - Students ↔ Courses with association tables
3. **One-to-One Relationships** - User ↔ Profile with unique foreign keys
4. **Relationship Loading** - Lazy vs eager loading strategies
5. **Advanced Queries** - Complex queries with joins and aggregations

## Key Takeaways

- **Foreign Keys**: Use `ForeignKey()` to establish relationships
- **Relationship Definitions**: Use `relationship()` with appropriate parameters
- **Association Tables**: Use `Table()` for many-to-many relationships
- **Loading Strategies**: Choose between lazy, joinedload, and subqueryload
- **Query Optimization**: Use joins and eager loading to reduce database queries
- **Bidirectional Relationships**: Use `back_populates` for proper two-way relationships

## Best Practices

- Always define relationships on both sides (bidirectional)
- Use appropriate loading strategies for performance
- Consider cascade options for data integrity
- Use association tables for many-to-many with additional data
- Test your relationships with various query patterns

Ready to practice? Move on to the exercise notebook!
